## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-01-16-47-18 +0000,nyt,U.S. airports report few delays as the shutdow...,https://www.nytimes.com/live/2025/10/01/us/gov...
1,2025-10-01-16-46-23 +0000,nypost,Mormons raise $180K for family of Michigan LDS...,https://nypost.com/2025/10/01/us-news/mormons-...
2,2025-10-01-16-45-19 +0000,wapo,Israel steps up Gaza City assault as Hamas wei...,https://www.washingtonpost.com/world/2025/10/0...
3,2025-10-01-16-44-58 +0000,nyt,Pope Leo Calls for Unity on Climate at a Divid...,https://www.nytimes.com/2025/10/01/world/europ...
4,2025-10-01-16-42-51 +0000,nypost,Queens takes crown as most bookish borough aft...,https://nypost.com/2025/10/01/us-news/nyc-libr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2453/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,48
4,shutdown,37
60,government,35
80,new,16
15,will,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...,184
244,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,142
327,2025-09-30-19-20-21 +0000,nypost,Trump threatens ‘irreversible’ actions if Dems...,https://nypost.com/2025/09/30/us-news/trump-th...,137
81,2025-10-01-13-50-00 +0000,wsj,The White House and lawmakers failed to reach ...,https://www.wsj.com/politics/policy/government...,121
311,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...,118


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,184,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...
311,70,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...
309,61,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
2,54,2025-10-01-16-45-19 +0000,wapo,Israel steps up Gaza City assault as Hamas wei...,https://www.washingtonpost.com/world/2025/10/0...
68,53,2025-10-01-14-17-00 +0000,wsj,Jeffrey Epstein never got kicked out of Wall S...,https://www.wsj.com/finance/jeffrey-epstein-we...
37,49,2025-10-01-15-20-56 +0000,latimes,Supreme Court puts off decision on whether Tru...,https://www.latimes.com/politics/story/2025-10...
244,49,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
145,38,2025-10-01-10-00-00 +0000,nypost,Howard Lutnick tells ‘Pod Force One’ ex-neighb...,https://nypost.com/2025/10/01/us-news/howard-l...
60,36,2025-10-01-14-35-35 +0000,latimes,Missing California woman found dead behind 'se...,https://www.latimes.com/california/story/2025-...
11,36,2025-10-01-16-37-00 +0000,wsj,The Trump administration said it is withholdin...,https://www.wsj.com/politics/policy/trump-admi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
